In [1]:
import nltk
import pandas as pd
import numpy as np
from lyricsgenius import Genius
from nltk.corpus import wordnet
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from flair.models import TextClassifier
from flair.data import Sentence
from langdetect import detect

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
client_access_token = "Z3u3rLvxxvbdSNOItyy-u0IZ60q-5JQqMj_sVIo2tyggpxAk3xPBGSBJMwKlQEeI"

/home/plasma/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
[nltk_data] Downloading package wordnet to /home/plasma/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/plasma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/plasma/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
df = pd.read_csv('muse_v3.csv').astype({'valence_tags': np.float32, 'arousal_tags':np.float32, 'dominance_tags':np.float32})
df = df.sample(frac=1).reset_index(drop=True) #randomize row order
df['lyrics'] = ""
df['lemma_lyrics'] = ""
df['textblob_sent'] = 0.0
df['textblob_sent_lemma'] = 0.0
df['vader_sent'] = 0.0
df['vader_sent_lemma'] = 0.0
df['flair_sent'] = 0.0
df['flair_sent_lemma'] = 0.0
df[:10]

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,lyrics,lemma_lyrics,textblob_sent,textblob_sent_lemma,vader_sent,vader_sent_lemma,flair_sent,flair_sent_lemma
0,https://www.last.fm/music/the%2bchemical%2bbro...,Surface To Air (Live),The Chemical Brothers,['bright'],2,6.040000,4.500000,4.780000,NaN,1jtHdm0zRRnS9wjHQa4k7S,electronic,,,0.0,0.0,0.0,0.0,0.0,0.0
1,https://www.last.fm/music/%25d0%25a2%25d0%25b5...,Пластмасса,Телевизор,['paranoid'],1,2.900000,5.090000,3.810000,cc59ce9e-16f1-47f0-ba56-57c78389f5bd,NaN,experimental,,,0.0,0.0,0.0,0.0,0.0,0.0
2,https://www.last.fm/music/aphex%2btwin/_/grey%...,Grey Stripe,Aphex Twin,"['dark', 'scary']",5,3.646667,3.828333,3.621667,NaN,NaN,ambient,,,0.0,0.0,0.0,0.0,0.0,0.0
3,https://www.last.fm/music/barry%2bwhite/_/love...,Love Serenade,Barry White,['sensual'],5,7.441518,5.318750,6.197053,b19e16bc-476b-4ed6-a257-85ee4346b7c3,46IYWXxeFPMLBf6imsuWKi,soul,,,0.0,0.0,0.0,0.0,0.0,0.0
4,https://www.last.fm/music/terre%2bthaemlitz/_/...,Hovering Glows,Terre Thaemlitz,['organic'],2,5.355000,3.770000,5.860000,27fae03f-e1b7-467a-9333-41554fda7a57,NaN,ambient,,,0.0,0.0,0.0,0.0,0.0,0.0
5,https://www.last.fm/music/sophie%2bellis-bexto...,Starlight,Sophie Ellis-Bextor,"['detached', 'bittersweet']",17,6.313590,4.560769,5.505384,298c550a-6edf-471f-830e-4a1cfcb7ea1e,6KnXEVNgJBr4cmC6kjAUGg,pop,,,0.0,0.0,0.0,0.0,0.0,0.0
6,https://www.last.fm/music/luther%2bvandross/_/...,Come Back,Luther Vandross,['slick'],3,5.526667,3.393333,5.686666,6669f2a6-87cb-4dc1-b73f-f10d99118539,6ykYnj6kpUsaUur53jMnhN,electronic,,,0.0,0.0,0.0,0.0,0.0,0.0
7,https://www.last.fm/music/asian%2bkung-fu%2bge...,ラストシーン,ASIAN KUNG-FU GENERATION,['sentimental'],2,3.135000,1.475000,3.365000,bdbb5174-d0f0-4928-a3c8-5010a430ec3b,NaN,rock,,,0.0,0.0,0.0,0.0,0.0,0.0
8,https://www.last.fm/music/jill%2bscott/_/brotha,Brotha,Jill Scott,['smooth'],5,6.533454,3.265455,6.228000,d0ac3ec2-0d3f-4430-ab63-7bcc91f34ca0,6IBWTv73pxGCRcl2gMW74m,soul,,,0.0,0.0,0.0,0.0,0.0,0.0
9,https://www.last.fm/music/hot%2bchip/_/the%2ba...,The Ass Attack,Hot Chip,['trippy'],2,5.580000,4.213333,4.693333,305080a1-6966-4921-82c7-72c5c8ef365b,1ej20TAi7DyF157rku23pR,indie,,,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
all_genres = set(df['genre'].unique())
'Rap' in all_genres

False

In [4]:
print("'")

'


In [5]:
sent = TextBlob("adshkbd ahjdba wdjh daih adhas djhad ajhsd asjdh asd aisdh asjhd ").sentiment
print(sent.polarity)


0.0


In [6]:
vader_analyzer = SentimentIntensityAnalyzer()
vader_analyzer.polarity_scores("I hate dogs so much")['compound']

-0.5719

In [7]:
genres = set()
for genre in df['genre']:
    genres.add(genre)
print(genres)

{nan, 'video game music', 'taiko', 'kora', 'swedish metal', 'french', 'fast melodic punk', 'environmental', 'synth-pop', 'glitch', 'native american', 'lute', 'jazz rock', 'death industrial', 'guitar', 'atmospheric black metal', 'field recording', 'pop rap', 'ragtime', 'jazz guitar', 'marching band', 'ragga jungle', 'symphonic metal', 'garage rock revival', 'gnawa', 'stoner metal', 'soundtrack', 'estonian metal', 'nasheed', 'frenchcore', 'russian indie', 'balafon', 'speedcore', 'contemporary country', 'progressive black metal', 'anti-folk', 'pop', 'turkish pop', 'britpop', 'avant-garde metal', 'power electronics', 'dark folk', 'smooth soul', 'carnaval', 'power noise', 'hammond organ', 'prank', 'bolero', 'contemporary folk', 'party', 'grime', 'avant-rock', 'uk pop', 'outlaw country', 'lovers rock', 'russian pop', 'digital hardcore', 'easy listening', 'funk metal', 'vocal house', 'doo-wop', '8-bit', 'industrial noise', 'enka', 'melodic metalcore', 'byzantine', 'melodic black metal', 'loun

In [3]:
def get_flair_sentiment(text, classifier):
    sentences = nltk.tokenize.sent_tokenize(text)
    sentiment = 0
    for sentence in sentences:
        flair_sentence = Sentence(sentence)
        classifier.predict(flair_sentence)
        dictionary = flair_sentence.to_dict()
        if dictionary['labels'][0]['value'] == 'POSITIVE':
            sentiment += dictionary['labels'][0]['confidence']
        else:
            sentiment += (-1*(dictionary['labels'][0]['confidence']))
    return sentiment/len(sentences)

In [4]:
def tag_to_wordnet(tag):
    if tag[0] == ('J') or tag[0] == 's':
        return wordnet.ADJ
    if tag[0] == ('V'):
        return wordnet.VERB
    if tag[0] == ('N'):
        return wordnet.NOUN
    if tag[0] == ('R'):
        return wordnet.ADV
    return None

In [5]:

def preprocess_lyrics(lyrics, lemmatizer):
    words = nltk.word_tokenize(lyrics)
    tagged_words = nltk.pos_tag(words)
    lemmatized_words = []
    for word,tag in tagged_words:
        if word:
            wordnet_tag = tag_to_wordnet(tag)
            if wordnet_tag:
                lemmatized_words.append(lemmatizer.lemmatize(word,wordnet_tag))
            else:
                lemmatized_words.append(lemmatizer.lemmatize(word))
    return " ".join(lemmatized_words)

In [6]:
print(preprocess_lyrics("I looked into the water and waved at the reflection!",nltk.stem.WordNetLemmatizer()))

I look into the water and wave at the reflection !


In [12]:
artist = df.iloc[0]['artist']
track = df.iloc[0]['track']

In [7]:
genius = Genius(client_access_token)
song = genius.search_song("The real slim Shady","Eminem")
print(song.lyrics)

Searching for "The real slim Shady" by Eminem...
Done.
[Intro]
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here...

[Verse 1]
Y'all act like you never seen a white person before
Jaws all on the floor like Pam like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (Agh!)
It's the return of the "Oh, wait, no way, you're kidding
He didn't just say what I think he did, did he?"
And Dr. Dre said… nothing, you idiots!
Dr. Dre's dead, he's locked in my basement (Ha ha!)
Feminist women love Eminem
"Chicka, chicka, chicka, Slim Shady, I'm sick of him
Look at him, walkin' around, grabbin' his you-know-what
Flippin' the you-know-who," "Yeah, but he's so cute though"
Yeah, I probably got a couple of screws up in my head loose
But no worse than what's goin' on in your parents

In [15]:
#test lemmatization
artist = df.iloc[0]['artist']
track = df.iloc[0]['track']
lyrics = genius.search_song(track,artist).lyrics
print(preprocess_lyrics(lyrics,nltk.stem.WordNetLemmatizer()))

Searching for "A Stimulant" by Dolce Triade...
Done.
Preparatory to anything else Mr Bloom brush off the great bulk of the shaving and hand Stephen the hat and ashplant and buck him up generally in orthodox Samaritan fashion , which he very badly need . His ( Stephen 's ) mind be not exactly what you would call wander but a bit unsteady and on his expressed desire for some beverage to drink Mr Bloom , in view of the hour it be and there be no pump of Vartry water available for their ablution , let alone drinking purpose , hit upon an expedient by suggest , off the reel , the propriety of the cabman 's shelter , a it be call , hardly a stonesthrow away near Butt Bridge , where they might hit upon some drinkable in the shape of a milk and soda or a mineral . But how to get there be the rub . For the nonce he be rather nonplussed but inasmuch a the duty plainly devolve upon him to take some measure on the subject he ponder suitable way and mean during which Stephen repeatedly yawn . So fa

In [8]:
lemmatizer = nltk.stem.WordNetLemmatizer()
vader_analyzer = SentimentIntensityAnalyzer()
flair_classifier = TextClassifier.load('en-sentiment')

2021-12-17 14:07:03,653 loading file /home/plasma/.flair/models/sentiment-en-mix-distillbert_4.pt


In [21]:

for i in range(0,5000):
    print(i)
    artist = df.iloc[i]['artist']
    track = df.iloc[i]['track']
    lyrics = ""
    try:
        lyrics = genius.search_song(track,artist).lyrics
    except BaseException: #if any error is thrown when getting the lyrics or lyrics not found
        lyrics = None
    if not lyrics or len(lyrics)> 15000 or detect(lyrics)!="en" :
        continue
    processed_lyrics = preprocess_lyrics(lyrics,lemmatizer)
    df.loc[i,'lemma_lyrics'] = processed_lyrics
    df.loc[i,'lyrics'] = lyrics
    #sentiment analysis part
    
    #TextBlob
    df.loc[i,'textblob_sent'] = TextBlob(lyrics).sentiment.polarity
    df.loc[i,'textblob_sent_lemma'] = TextBlob(processed_lyrics).sentiment.polarity #polarity represents how negative or positive the text is
    #Vader
    df.loc[i,'vader_sent'] = vader_analyzer.polarity_scores(lyrics)['compound']
    df.loc[i,'vader_sent_lemma']  = vader_analyzer.polarity_scores(processed_lyrics)['compound']
    #Flair
    df.loc[i,'flair_sent'] = get_flair_sentiment(lyrics,flair_classifier)
    df.loc[i,'flair_sent_lemma'] = get_flair_sentiment(processed_lyrics,flair_classifier)

print(df[:4])

0
Searching for "Posed By Models" by Young Marble Giants...
Done.
1
Searching for "small mr man pants (w/ andrew broder and orvar smarason)" by Odd Nosdam...
No results found for: 'small mr man pants (w/ andrew broder and orvar smarason) Odd Nosdam'
2
Searching for "Introduction" by Nick Drake...
Specified song does not contain lyrics. Rejecting.
3
Searching for "O Holy Jesus" by The Cathedral Singers...
Done.
4
Searching for "Ушла" by Аукцыон...
Done.
5
Searching for "Melodica (original mix)" by Leama...
Done.
6
Searching for "Laura" by Oscar Peterson...
Done.
7
Searching for "10 - Naive (Big Weekend)" by The Kooks...
No results found for: '10 - Naive (Big Weekend) The Kooks'
8
Searching for "Stay the Night" by James Blunt...
Done.
9
Searching for "The Gambler" by fun....
Done.
10
Searching for "Widdlecombe Fair (Caister Norfolk)" by Tom Brown...
No results found for: 'Widdlecombe Fair (Caister Norfolk) Tom Brown'
11
Searching for "Forever and Ever" by Project Divinity...
Done.
12
Sea

KeyboardInterrupt: 

In [127]:
text = "As Twitter offers a fertile ground for expressing different thoughts and opinions, it can be seen as a valuable tool for sentiment analysis. Furthermore, properly identified reviews present a baseline of information as an input to different systems, such as e-learning systems, decision support systems etc."
processed_text = preprocess_lyrics(text,lemmatizer)
sentences = nltk.tokenize.sent_tokenize(text)
processed_sentences = nltk.tokenize.sent_tokenize(processed_text)
print(processed_sentences)

['As Twitter offer a fertile ground for express different thought and opinion , it can be see a a valuable tool for sentiment analysis .', 'Furthermore , properly identify review present a baseline of information a an input to different system , such a e-learning system , decision support system etc .']


In [168]:
df.loc[0]

lastfm_url                https://www.last.fm/music/eminem/_/%2527till%2...
track                                                      'Till I Collapse
artist                                                               Eminem
seeds                                                        ['aggressive']
number_of_emotion_tags                                                    6
valence_tags                                                           4.55
arousal_tags                                                       5.273125
dominance_tags                                                     5.690625
mbid                                   cab93def-26c5-4fb0-bedd-26ec4c1619e1
spotify_id                                           4xkOaSrkexMciUUogZKVTS
genre                                                                   rap
lyrics                                                                     
lemma_lyrics                                                               
textblob_sen

In [170]:
i = 0
artist = df.iloc[i]['artist']
track = df.iloc[i]['track']
lyrics = ""
try:
    lyrics = genius.search_song(track,artist).lyrics
except BaseException: #if any error is thrown when getting the lyrics or lyrics not found
    lyrics = None
if not lyrics:
    print("No lyrics")
processed_lyrics = preprocess_lyrics(lyrics,lemmatizer)
df.loc[i,'lemma_lyrics'] = processed_lyrics
#sentiment analysis part
    
#TextBlob
df.loc[i,'textblob_sent'] = TextBlob(lyrics).sentiment.polarity
df.loc[i,'textblob_sent_lemma'] = TextBlob(processed_lyrics).sentiment.polarity #polarity represents how negative or positive the text is
#Vader
df.loc[i,'vader_sent'] = vader_analyzer.polarity_scores(lyrics)['compound']
df.loc[i,'vader_sent_lemma']  = vader_analyzer.polarity_scores(processed_lyrics)['compound']
#Flair
df.loc[i,'flair_sent'] = get_flair_sentiment(lyrics,flair_classifier)
df.loc[i,'flair_sent_lemma'] = get_flair_sentiment(processed_lyrics,flair_classifier)

Searching for "'Till I Collapse" by Eminem...
Done.


In [9]:
def calculate_sentiment_genres(genres):
    genres = [genre.lower() for genre in genres]
    genre_set= set(genres)
    for i in range(0,90000):
        if df.iloc[i]['genre'] not in genre_set or df.iloc[i]['lyrics'] != "":
            continue
        print(i)
        artist = df.iloc[i]['artist']
        track = df.iloc[i]['track']
        lyrics = ""
        try:
            lyrics = genius.search_song(track,artist).lyrics
        except BaseException: #if any error is thrown when getting the lyrics or lyrics not found
            lyrics = None
        if not lyrics or len(lyrics)> 15000 or detect(lyrics)!="en" :
            continue
        processed_lyrics = preprocess_lyrics(lyrics,lemmatizer)
        df.loc[i,'lemma_lyrics'] = processed_lyrics
        df.loc[i,'lyrics'] = lyrics
    #sentiment analysis part
    
        #TextBlob
        df.loc[i,'textblob_sent'] = TextBlob(lyrics).sentiment.polarity
        df.loc[i,'textblob_sent_lemma'] = TextBlob(processed_lyrics).sentiment.polarity #polarity represents how negative or positive the text is
        #Vader
        df.loc[i,'vader_sent'] = vader_analyzer.polarity_scores(lyrics)['compound']
        df.loc[i,'vader_sent_lemma']  = vader_analyzer.polarity_scores(processed_lyrics)['compound']
        #Flair
        df.loc[i,'flair_sent'] = get_flair_sentiment(lyrics,flair_classifier)
        df.loc[i,'flair_sent_lemma'] = get_flair_sentiment(processed_lyrics,flair_classifier)

def calculate_sentiment_artists(artists):
    artist_set= set(artists)
    for i in range(0,90000):
        if df.iloc[i]['artist'] not in artist_set or df.iloc[i]['lyrics'] != "":
            continue
        print(i)
        artist = df.iloc[i]['artist']
        track = df.iloc[i]['track']
        lyrics = ""
        try:
            lyrics = genius.search_song(track,artist).lyrics
        except BaseException: #if any error is thrown when getting the lyrics or lyrics not found
            lyrics = None
        if not lyrics or len(lyrics)> 15000 or detect(lyrics)!="en" :
            continue
        processed_lyrics = preprocess_lyrics(lyrics,lemmatizer)
        df.loc[i,'lemma_lyrics'] = processed_lyrics
        df.loc[i,'lyrics'] = lyrics
    #sentiment analysis part
    
        #TextBlob
        df.loc[i,'textblob_sent'] = TextBlob(lyrics).sentiment.polarity
        df.loc[i,'textblob_sent_lemma'] = TextBlob(processed_lyrics).sentiment.polarity #polarity represents how negative or positive the text is
        #Vader
        df.loc[i,'vader_sent'] = vader_analyzer.polarity_scores(lyrics)['compound']
        df.loc[i,'vader_sent_lemma']  = vader_analyzer.polarity_scores(processed_lyrics)['compound']
        #Flair
        df.loc[i,'flair_sent'] = get_flair_sentiment(lyrics,flair_classifier)
        df.loc[i,'flair_sent_lemma'] = get_flair_sentiment(processed_lyrics,flair_classifier)

In [11]:
calculate_sentiment_genres(["hip-hop","contemporary jazz"])
#"country", "hip-hop","contemporary jazz", "blues", "rock", "experimental", "metal","r&b","folk","rap/"

25
Searching for "You Know Who It Is (Black&Yellow)" by Randy Rocket...
Done.
103
Searching for "Forever" by Drake feat. Kanye West, Lil Wayne & Eminem...
Done.
163
Searching for "Announcement" by Common...
Done.
215
Searching for "I Against I (feat. Planetary of Outerspace)" by Jedi Mind Tricks...
No results found for: 'I Against I (feat. Planetary of Outerspace) Jedi Mind Tricks'
291
Searching for "Intro" by Common...
Specified song does not contain lyrics. Rejecting.
299
Searching for "Diddy Bop" by Tyga...
Done.
469
Searching for "I-Deez" by 8 Bit...
Done.
545
Searching for "I Like Dat" by Masta Ace...
Done.
548
Searching for "Can't Win For Losing (album)" by Little Brother...
Done.
636
Searching for "OMG!" by Marteria...
Done.
648
Searching for "Champion Requiem" by Mos Def...
Done.
677
Searching for "Addiction" by Kanye West...
Done.
713
Searching for "Wool" by Earl Sweatshirt...
Done.
731
Searching for "Titel 1" by Hardman...
Done.
802
Searching for "Rape" by Pharoahe Monch...
D

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Done.



KeyboardInterrupt



In [180]:
df['artist'].nunique()

26012

In [181]:
df.iloc[0:10]

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,lyrics,lemma_lyrics,textblob_sent,textblob_sent_lemma,vader_sent,vader_sent_lemma,flair_sent,flair_sent_lemma
0,https://www.last.fm/music/eminem/_/%2527till%2...,'Till I Collapse,Eminem,['aggressive'],6,4.550000,5.273125,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap,,[ Intro : Eminem ] 'Cause sometimes you just f...,-0.045686,-0.012063,-0.9944,-0.9951,-0.159126,-0.262268
1,https://www.last.fm/music/metallica/_/st.%2banger,St. Anger,Metallica,['aggressive'],8,3.710000,5.833000,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal,,[ Verse ] Saint Anger 'round my neck Saint Ang...,-0.321575,-0.297039,-0.9992,-0.9993,-0.095738,-0.072000
2,https://www.last.fm/music/rick%2bross/_/speedi...,Speedin',Rick Ross,['aggressive'],1,3.080000,5.870000,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap,,[ Intro : Rick Ross ] Legendary Runners You kn...,0.133909,0.133909,0.9480,0.9737,-0.999637,-0.999840
3,https://www.last.fm/music/m.i.a./_/bamboo%2bbanga,Bamboo Banga,M.I.A.,"['aggressive', 'fun', 'sexy', 'energetic']",13,6.555071,5.537214,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop,,"[ Intro ] Road runner , road runner Going hund...",-0.062335,-0.065536,0.9701,0.9812,0.562925,0.640141
4,https://www.last.fm/music/dope/_/die%2bmf%2bdie,Die MF Die,Dope,['aggressive'],7,3.771177,5.348235,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal,,Die ! [ Verse ] I do n't need your forgiveness...,-0.481035,-0.481035,-0.9998,-0.9998,-0.702031,-0.641719
5,https://www.last.fm/music/drowning%2bpool/_/st...,Step Up,Drowning Pool,['aggressive'],9,2.971389,5.537500,4.726389,49e7b4d2-3772-4301-ba25-3cc46ceb342e,4Q1w4Ryyi8KNxxaFlOQClK,metal,,"[ Intro ] One , two , three , go ! [ Verse 1 ]...",-0.014527,-0.037016,0.9575,0.9563,0.297424,0.315605
6,https://www.last.fm/music/kanye%2bwest/_/feedback,Feedback,Kanye West,['aggressive'],1,3.080000,5.870000,5.490000,NaN,49fT6owWuknekShh9utsjv,hip-hop,,"[ Chorus ] Ayy , ya hear about the good news ?...",0.141057,0.162426,0.9658,0.9732,-0.112662,0.036590
7,https://www.last.fm/music/deftones/_/7%2bwords,7 Words,Deftones,"['aggressive', 'angry']",10,3.807121,5.473939,4.729091,1a826083-5585-445f-a708-415dc90aa050,6DoXuH326aAYEN8CnlLmhP,nu metal,,"[ Verse 1 ] I 'll never be the same , break de...",-0.188605,-0.179162,-0.9999,-0.9999,-0.528519,-0.451526
8,https://www.last.fm/music/fiona%2bapple/_/limp,Limp,Fiona Apple,"['aggressive', 'angry', 'bitter']",20,3.737211,5.610204,4.626735,4435982c-b83e-4daa-af2b-9f3430036bb7,104YdibC7VQy78xAVmgRYr,singer-songwriter,,"[ Verse 1 ] You wan na make me sick , you wan ...",-0.111548,-0.119589,-0.9969,-0.9853,-0.081627,-0.866962
9,https://www.last.fm/music/metallica/_/sweet%2b...,Sweet Amber,Metallica,['aggressive'],4,3.582759,5.757241,5.340000,fe1cc051-faa7-4953-b331-f6196cd3ddae,5fU6qjmD38P90BMsuqpiuU,metal,,[ Verse 1 ] Wash your back so you do n't stab ...,0.242893,0.231502,0.9973,0.9972,0.787943,0.341907


In [182]:
df.to_csv('partialcsv.csv', index = False)

In [195]:
df.iloc[25:30]

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,lyrics,lemma_lyrics,textblob_sent,textblob_sent_lemma,vader_sent,vader_sent_lemma,flair_sent,flair_sent_lemma
25,https://www.last.fm/music/a%2bflock%2bof%2bsea...,Standing in the Doorway,A Flock of Seagulls,['exciting'],1,7.950000,6.950000,7.21000,9514ad3b-b995-4ac9-bbe2-0ca0a2973df2,4pkEoiyCaOtwxUL8C6IGkS,new wave,I can see you standing outside\nSee your shado...,I can see you stand outside See your shadow on...,-0.058333,-0.058333,0.6124,0.6124,-0.590647,-0.691504
26,https://www.last.fm/music/tiger%2blou/_/nation...,National Ave,Tiger Lou,['reflective'],3,5.400000,3.060000,6.08000,72e814fd-a2f5-4049-9e36-451811f487f0,3qvA3ZBRYgZ6ci2YItP4Ye,indie,Where do you wanna go?\nI wanna go home\nHome ...,Where do you wan na go ? I wan na go home Home...,0.250000,-0.087500,-0.2500,-0.2500,0.780610,0.751677
27,https://www.last.fm/music/eufonius/_/wish,wish,eufonius,['pure'],1,6.800000,4.050000,6.44000,941100ab-9e8a-4f7d-9218-42aa6f543e0c,NaN,j-pop,CHAPTER TWO\n\nTHE MYSTERIES OF ALBERTINE — TH...,CHAPTER TWO THE MYSTERIES OF ALBERTINE — THE G...,0.101581,0.101506,0.0000,0.0000,0.000000,0.000000
28,https://www.last.fm/music/jon%2bhassell/_/char...,Charm (Over Burundi Cloud),Jon Hassell,['sophisticated'],8,6.163750,3.881250,6.02375,NaN,NaN,soundtrack,,,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000
29,https://www.last.fm/music/hard-fi/_/help%2bme%...,Help Me Please,Hard-Fi,"['romantic', 'soft']",6,5.697227,3.487815,5.83395,NaN,NaN,indie,,,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000


In [32]:
print(all_genres)

{nan, 'video game music', 'taiko', 'kora', 'swedish metal', 'french', 'fast melodic punk', 'environmental', 'synth-pop', 'glitch', 'native american', 'lute', 'jazz rock', 'death industrial', 'guitar', 'atmospheric black metal', 'jazz guitar', 'field recording', 'ragtime', 'pop rap', 'marching band', 'ragga jungle', 'symphonic metal', 'gnawa', 'garage rock revival', 'stoner metal', 'soundtrack', 'estonian metal', 'nasheed', 'russian indie', 'frenchcore', 'balafon', 'speedcore', 'contemporary country', 'progressive black metal', 'anti-folk', 'pop', 'turkish pop', 'britpop', 'avant-garde metal', 'power electronics', 'dark folk', 'smooth soul', 'carnaval', 'power noise', 'hammond organ', 'prank', 'bolero', 'contemporary folk', 'party', 'grime', 'uk pop', 'avant-rock', 'outlaw country', 'lovers rock', 'russian pop', 'digital hardcore', 'easy listening', 'funk metal', 'vocal house', 'doo-wop', '8-bit', 'industrial noise', 'enka', 'melodic metalcore', 'byzantine', 'melodic black metal', 'loun